In [1]:
!pip install apimoex

In [2]:
import sys
import apimoex
import pandas as pd
import requests
import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.model_selection import train_test_split
from scipy.stats import shapiro, kstest
from statsmodels.tsa.stattools import adfuller

In [3]:
# Последняя рабочая неделя, для сбора статистики
monday = datetime.datetime(year=2024, month=12, day=16)
tuesday = datetime.datetime(year=2024, month=12, day=17)
wednesday = datetime.datetime(year=2024, month=12, day=18)
thursday = datetime.datetime(year=2024, month=12, day=19)
friday = datetime.datetime(year=2024, month=12, day=20)
week = [monday, tuesday, wednesday, thursday, friday]

In [5]:
# Рассматриваемые тикеры для статистик
tickers = ["SBER", "VTBR", "ROSN", "GOLD", "LKOH"]
extra_tickers = ["MOEXFN", "MOEXCN", "TRUR", ]

In [24]:
seven = datetime.timedelta(days=7)
data = {}
friday = datetime.datetime.now()

with requests.Session() as session:
    for i in tickers:
        for weeks in range(36):
            dt = friday - weeks * seven
            candles = apimoex.get_market_candles(session, i, 1, str(dt - datetime.timedelta(days=5))[:-9], str(dt)[:-9])
            dc = pd.DataFrame(candles)
            if i not in data:
                data[i] = [dc]
            else:
                data[i].append(dc)



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [25]:
for key in data:
    data[key] = pd.concat(data[key], ignore_index=True)
data["VTBR"].head()

,begin,open,close,high,low,value,volume
0,2025-01-08 10:03:00,78.17,78.27,78.28,78.11,6760424.91,86421
1,2025-01-08 10:04:00,78.27,78.32,78.37,78.22,4270218.40,54533
2,2025-01-08 10:05:00,78.33,78.66,78.67,78.33,14300586.66,182108
3,2025-01-08 10:06:00,78.66,78.43,78.66,78.40,7769757.60,98915
4,2025-01-08 10:07:00,78.41,78.43,78.49,78.40,2634792.12,33582


In [27]:
import google.colab.drive
import pickle
google.colab.drive.mount('/content/drive')


def save_data_as_pickle(data, filename):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)

def load_data_from_pickle(filename):
    with open(filename, 'rb') as f:
        data = pickle.load(f)
    return data

drive_path = "/content/drive/My Drive/candles_data.pkl"
save_data_as_pickle(data, drive_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
import os

if os.path.exists(drive_path):
    data = load_data_from_pickle(drive_path)
else:
    print(f"Файл {drive_path} не найден.")

data.keys()

dict_keys(['SBER', 'VTBR', 'ROSN', 'GOLD', 'LKOH'])

In [43]:
print(f'Количество тикеров: {len(data)}')
print(f'Количество свечей: {len(data[tickers[0]])}')

def time_split(df_dict, train_ratio=0.7, val_ratio=0.2):
    train_dict = {}
    val_dict = {}
    test_dict = {}

    for key, df in df_dict.items():
        n = len(df)
        train_end = int(n * train_ratio)
        val_end = int(n * (train_ratio + val_ratio))

        df_train = df.iloc[:train_end].reset_index(drop=True)
        df_val   = df.iloc[train_end:val_end].reset_index(drop=True)
        df_test  = df.iloc[val_end:].reset_index(drop=True)

        train_dict[key] = df_train
        val_dict[key] = df_val
        test_dict[key] = df_test

    return train_dict, val_dict, test_dict


Количество тикеров: 5
Количество свечей: 85901


In [36]:
!pip install stable-baselines3
!pip install shimmy

In [55]:
import gym
import numpy as np
import pandas as pd

from gym import spaces
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv


def prepare_data(data_dict, ticker, time_steps=100):
    df = data_dict[ticker]
    # df = pd.concat(df_list, ignore_index=True) # If not zipped upper

    df = df.sort_values(by=['begin'], ascending=True).reset_index(drop=True)

    return df

class MoexTradingEnv(gym.Env):
    """
    A custom trading environment for a single ticker's time series data.
    """
    def __init__(self, df: pd.DataFrame, window_size: int = 10, alpha: float = 0.0001, volume=1e5):
        super(MoexTradingEnv, self).__init__()

        self.df = df.reset_index(drop=True)
        self.window_size = window_size
        self.alpha = alpha  # penalty coefficient

        # Define action space: {0: hold, 1: open long, 2: close long, 3: open short, 4: close short}
        self.action_space = spaces.Discrete(5)

        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(window_size * 5,), dtype=np.float32
        )

        # Internal variables to track state
        self.current_step = 0
        self.position = None  # can be 'long' or 'short' or None
        self.entry_price = 0.0
        self.entry_step = 0  # track how long the position has been held
        self.volume = volume
        self.eta = 0

    def _get_observation(self):
        """
        Returns the last `window_size` candles as a flattened array:
        [open, high, low, close, volume, ... repeated window_size times ...].
        """
        start = self.current_step - self.window_size + 1
        if start < 0:
            # If we are at the beginning, pad with the first row
            pad_size = abs(start)
            front_pad = np.tile(self.df.iloc[0][['open','high','low','close','volume']].values, (pad_size,1))
            obs_data = pd.concat([
                pd.DataFrame(front_pad, columns=['open','high','low','close','volume']),
                self.df.iloc[:self.current_step+1][['open','high','low','close','volume']]
            ], ignore_index=True)
        else:
            obs_data = self.df.iloc[start:self.current_step+1][['open','high','low','close','volume']]

        obs = obs_data.values.flatten()
        return obs.astype(np.float32)

    def _calculate_reward(self, action):
        reward = 0.0
        current_price = self.df.iloc[self.current_step]['close']

        if action == 1:  # open long
            if self.position is None:
                self.position = 'long'
                self.entry_price = current_price
                self.entry_step = self.current_step

        elif action == 2:  # close long
            if self.position == 'long':
                # Profit is (sell_price - buy_price)
                reward = (current_price - self.entry_price) * self.volume
                # Subtract our holding penalty
                hold_time = (self.current_step - self.entry_step)
                reward -= (hold_time * self.alpha)

                self.position = None

        elif action == 3:  # open short
            if self.position is None:
                self.position = 'short'
                self.entry_price = current_price
                self.entry_step = self.current_step

        elif action == 4:  # close short
            if self.position == 'short':
                # Profit is (buy_price - sell_price)
                reward = (self.entry_price - current_price) * self.volume
                # Subtract our holding penalty
                hold_time = (self.current_step - self.entry_step)
                reward -= (hold_time * self.alpha)

                self.position = None

        # If action == 0 or the position is still open, we can give no immediate reward
        # but we might incorporate a negative drift penalty for open position if desired.
        if action == 0 and self.position is None:
            reward -= self.alpha * self.eta

        return reward

    def step(self, action):
        reward = self._calculate_reward(action)

        self.current_step += 1
        done = False
        if self.current_step >= len(self.df) - 1:
            done = True

        obs = self._get_observation()
        info = {}

        return obs, reward, done, info

    def reset(self):
        self.current_step = 0
        self.position = None
        self.entry_price = 0.0
        self.entry_step = 0

        return self._get_observation()


def train_vanilla_rl(data_dict, ticker, window_size=10, alpha=0.0001, total_timesteps=10000):
    df = prepare_data(data_dict, ticker)
    env = MoexTradingEnv(df, window_size=window_size, alpha=alpha)
    vec_env = DummyVecEnv([lambda: env])
    model = PPO("MlpPolicy", vec_env, verbose=1, n_epochs=20)
    model.learn(total_timesteps=total_timesteps)
    return model



In [56]:
ticker = "VTBR"

df_train, df_val, df_test = time_split(data, train_ratio=0.7, val_ratio=0.2)
print("Train size:", len(df_train["VTBR"]))
print("Val size:", len(df_val["VTBR"]))
print("Test size:", len(df_test["VTBR"]))



model = train_vanilla_rl(df_train, "VTBR", window_size=10, alpha=0.0002, total_timesteps=50000)


Train size: 58416
Val size: 16690
Test size: 8346
Using cpu device


/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


-----------------------------
| time/              |      |
|    fps             | 438  |
|    iterations      | 1    |
|    time_elapsed    | 4    |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 364          |
|    iterations           | 2            |
|    time_elapsed         | 11           |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0006259612 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.61        |
|    explained_variance   | 6.56e-07     |
|    learning_rate        | 0.0003       |
|    loss                 | 3.88e+08     |
|    n_updates            | 20           |
|    policy_gradient_loss | -0.00238     |
|    value_loss           | 3.87e+08     |
------------------------------------------
----------------

In [57]:
def evaluate_agent(model, df_val, max_steps=10_000):
    env = MoexTradingEnv(df_val, window_size=10, alpha=0.0000)
    obs = env.reset()
    done = False
    total_reward = 0.0
    steps = 0

    while not done and steps < max_steps:
        # predict action
        action, _states = model.predict(obs, deterministic=True)
        obs, reward, done, info = env.step(action)
        #print(reward)
        total_reward += reward
        steps += 1

    return total_reward

val_profit = evaluate_agent(model, df_val["VTBR"])
print(f"Validation total profit: {val_profit:.2f}")




/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Validation total profit: 336000.00
